In [1]:
import ngsolve as ngs
from do import solve
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from copy import deepcopy

levels = 1
deg = 3
degdif = 1

######################################################
# from bhdata import BHCurves
# fun_dw  = BHCurves(4)
# fun_w   = fun_dw.Integrate()
# fun_ddw = fun_dw.Differentiate()
######################################################
from bhdata import Brauer, BrauerCut
fun_w, fun_dw, fun_ddw  = Brauer()
######################################################

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
######################################################

from createGeom import makeGeo
geoOCC = makeGeo()

Applying a current of 196.88 Amps


In [2]:

meshes = []; its = []; errors = []

for i in range(levels):

    print("#####################################################")
    print(f"# Level {i}")
    print("#####################################################")

    ngmesh = geoOCC.GenerateMesh()
    mesh = ngs.Mesh(ngmesh)

    for j in range(i):
        mesh.Refine()
    mesh.Curve(2)
    
    with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
    
    HCurl_0 = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face')
    L2_0 = ngs.L2(mesh, dim = 3, order = deg-1)
    A0 = ngs.GridFunction(HCurl_0)
    A0, it = solve(HCurl_0, A0, mesh, deg, J, fun_w, fun_dw, fun_ddw, linear, nonlinear)
    its.append(it)
    B0 = ngs.GridFunction(L2_0, nested = True)
    with ngs.TaskManager(): B0.Set(ngs.curl(A0), bonus_intorder = 10)

    
    HCurl_1 = ngs.HCurl(mesh, order = deg+degdif, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face')
    L2_1 = ngs.L2(mesh, dim = 3, order = deg+degdif-1)
    A0 = ngs.GridFunction(HCurl_1)
    A0, it = solve(HCurl_1, A0, mesh, deg+degdif, J, fun_w, fun_dw, fun_ddw, linear, nonlinear)
    its.append(it)
    B1 = ngs.GridFunction(L2_1, nested = True)
    with ngs.TaskManager(): B1.Set(ngs.curl(A0), bonus_intorder = 10)

    with ngs.TaskManager(): error = ngs.Integrate((B0-B1)**2, mesh, order = 3*(deg+degdif))**(1/2)/\
                                    ngs.Integrate((B1)**2, mesh, order = 3*(deg+degdif))**(1/2)
    print(error)
    errors.append(error)
    
print(its)

#####################################################
# Level 0
#####################################################
Using 3D mesh with ne= 850 elements and nv= 265 points and  13442 DOFs.
 
Iter:  1 | assem : 0.18s | CG took 0.07s with    0 iterations | alpha : 1.00 | energy = 0.0000000015 | relres = 1.00e+00 |
Iter:  2 | assem : 0.26s | CG took 0.06s with    0 iterations | alpha : 1.00 | energy = -0.1039725754 | relres = 2.19e+00 |
Iter:  3 | assem : 0.17s | CG took 0.06s with    0 iterations | alpha : 1.00 | energy = -0.1442354716 | relres = 8.54e-01 |
Iter:  4 | assem : 0.18s | CG took 0.06s with    0 iterations | alpha : 1.00 | energy = -0.1609767874 | relres = 3.24e-01 |
Iter:  5 | assem : 0.24s | CG took 0.07s with    0 iterations | alpha : 1.00 | energy = -0.1663037723 | relres = 1.18e-01 |
Iter:  6 | assem : 0.21s | CG took 0.08s with    0 iterations | alpha : 0.25 | energy = -0.1674162354 | relres = 4.02e-02 |
Iter:  7 | assem : 0.27s | CG took 0.06s with    0 iterations | a

In [3]:
import numpy as np
a = np.array(errors)
print(np.log2(a[:-1]/a[1:]))

[]


In [4]:
clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)
with ngs.TaskManager(): Draw(B1, mesh, clipping = clipping)
with ngs.TaskManager(): Draw(ngs.log(ngs.Norm(B0-B1)), mesh, clipping = clipping)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [5]:
stop

NameError: name 'stop' is not defined

In [ ]:
# meshes = []; Bs = []; Bps = []; its = [];

# for i in range(levels):
#     ngmesh = geoOCC.GenerateMesh()
#     mesh = ngs.Mesh(ngmesh)

#     for j in range(i):
#         mesh.Refine()
#     mesh.Curve(3)

#     with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
#     HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
#     V3L2 = ngs.L2(mesh, dim = 3, order = deg-1, autoupdate = True)
#     A0 = ngs.GridFunction(HCurl)
#     A0, it = solve(HCurl, A0, mesh,deg, J, fun_w, fun_dw, fun_ddw)
#     its.append(it)

#     B = ngs.GridFunction(V3L2, nested = True)
#     with ngs.TaskManager(): B.Set(ngs.curl(A0))
#     Bs.append(B)

#     if i>0:
#         V3L2 = ngs.L2(mesh, dim = 3, order = deg-1)
#         Bp = ngs.GridFunction(V3L2)
#         with ngs.TaskManager(): Bp.Set(Bs[i-1])
#         Bps.append(Bp)
#         with ngs.TaskManager(): error = ngs.Integrate((B-Bp)**2, mesh)**(1/2)/\
#                                         ngs.Integrate((B)**2, mesh)**(1/2)
#         print(error)
        

#     # ######################################################
#     # # Refine mesh, apply prolongation
#     # ######################################################

#     # ngmesh2 = geoOCC.GenerateMesh()
#     # mesh2 = ngs.Mesh(ngmesh2)

#     # for j in range(i+1):
#     #     mesh2.Refine()
#     # mesh2.Curve()

#     # V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
#     # Bp = ngs.GridFunction(V3L2)
#     # Bp.Set(B)
#     # Bps.append(B)


# print(its)


In [ ]:
import numpy as np
a = np.array([5.332232056354972e-06, 2.246711173326673e-06, 6.508678547245244e-07])
np.log2(a[:-1]/a[1:])

In [ ]:
######################################################
# Compute first solution
######################################################

ngmesh = geoOCC.GenerateMesh()
mesh = ngs.Mesh(ngmesh)
# mesh.Refine()
mesh.Curve(3)

levels = 2
deg = 1

J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))

HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
V3L2 = ngs.L2(mesh, dim=3, order = deg-1, autoupdate = True)

A0 = ngs.GridFunction(HCurl)
A0, it0 = solve(HCurl,A0,mesh,deg,J,fun_w,fun_dw,fun_ddw)

B0 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B0.Set(ngs.curl(A0))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)

######################################################
# Refine mesh, apply prolongation
######################################################

ngmesh2 = geoOCC.GenerateMesh()
mesh2 = ngs.Mesh(ngmesh2)

mesh2.Refine()
# mesh2.Refine()
mesh2.Curve(3)

V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
B01 = ngs.GridFunction(V3L2)
B01.Set(B0)

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01, mesh2, clipping = clipping)

######################################################
# Compute second solution
######################################################

HCurl = ngs.HCurl(mesh2, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)

A1 = ngs.GridFunction(HCurl)
A1, it1 = solve(HCurl,A1,mesh2,deg,J,fun_w,fun_dw,fun_ddw)

B1 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B1.Set(ngs.curl(A1))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B1, mesh2, clipping = clipping)


######################################################
# Compute error
######################################################

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01-B1, mesh2, clipping = clipping)

error = ngs.Integrate((B01-B1)**2, mesh)*(1/2)
error


In [ ]:
ngmesh2 = geoOCC.GenerateMesh()
mesh2 = ngs.Mesh(ngmesh2)

mesh2.Refine()
mesh2.Refine()
mesh2.Curve(3)

V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
B01 = ngs.GridFunction(V3L2)
B01.Set(B0)

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01, mesh2, clipping = clipping)

In [ ]:
HCurl = ngs.HCurl(mesh2, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
A1 = ngs.GridFunction(HCurl)
A1, it1 = solve(HCurl,A1,mesh2,deg,J,fun_w,fun_dw,fun_ddw)

V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
B1 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B1.Set(ngs.curl(A1))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B1, mesh2, clipping = clipping)

In [ ]:
error = ngs.Integrate((B01-B1)**2, mesh)*(1/2)
error

In [ ]:
help(ngs.)